In [ ]:
from mt import *

In [ ]:
test()

In [ ]:
# The directory containing the images
data_dir = '/Volumes/martin/200116_tandem_retina_explants/27-T3/ori/14/bgsub'

# glob string for images
im_glob_dapi = os.path.join(data_dir, '200116_tandem_bAA27_T3_1_w1*')
im_list_dapi = sorted(glob.glob(im_glob_dapi))
im_glob_cfp = os.path.join(data_dir, '200116_tandem_bAA27_T3_1_w2*')
im_list_cfp = sorted(glob.glob(im_glob_cfp))
im_glob_r = os.path.join(data_dir, '200116_tandem_bAA27_T3_1_w3*')
im_list_r = sorted(glob.glob(im_glob_r))
im_glob_fr = os.path.join(data_dir, '200116_tandem_bAA27_T3_1_w4*')
im_list_fr = sorted(glob.glob(im_glob_fr))

im_list_fr

In [ ]:
cfp_list = []
r_list = []
fr_list = []
label_image_cfp_small2_list = []
label_image_cfp_small_dil_list = []
label_image_fr_small_list = []
label_image_r_small_list = []
label_image_pos_nuc_list = []
df_pos_nuc_list = []
df_counts_zero_list = []
spikes_list = []
silenced_list = []
any_barcode_list = []
one_barcode_list = []
one_plus_barcode_list = []

In [ ]:
def codots_function():
    
    for i, (d, c, r, fr) in enumerate(zip(im_list_dapi, im_list_cfp, im_list_r, im_list_fr)):

        print('')
        print('this is index:', i)

        # read the file
        dapi_raw = skimage.io.imread(d)
        cfp_raw = skimage.io.imread(c)
        r_raw = skimage.io.imread(r)
        fr_raw = skimage.io.imread(fr)

        dapi, cfp, r, fr = dapi_raw, cfp_raw, r_raw, fr_raw

        # Adjust image
        cfp_adj = imadjust(cfp, lower_bound=0.1, upper_bound=99.9)
        r_adj = imadjust(r, lower_bound=0.1, upper_bound=99.9)
        fr_adj = imadjust(fr, lower_bound=0.1, upper_bound=99.9)

        # Register image
        fr_adj = register_images(r_adj, fr_adj)

        cfp_list.append(cfp_adj)
        r_list.append(r_adj)
        fr_list.append(fr_adj)
        
        # Apply gaussian filter
        cfp_adj_g, r_adj_g, fr_adj_g = filt_gauss(cfp_adj, r_adj, fr_adj, 3)

        # Threshold using multiotsu
        print(threshold_multiotsu(cfp_adj_g, classes=4))
        cfp_mo0 = cfp_adj_g > threshold_multiotsu(cfp_adj_g, classes=4)[0]
        cfp_mo1 = cfp_adj_g > threshold_multiotsu(cfp_adj_g, classes=4)[2]

        print(threshold_multiotsu(fr_adj_g, classes=3))
        fr_mo0 = fr_adj_g > threshold_multiotsu(fr_adj_g)[0]
        fr_mo1 = fr_adj_g > threshold_multiotsu(fr_adj_g)[1]

        print(threshold_multiotsu(r_adj_g, classes=4))
        r_mo0 = r_adj_g > threshold_multiotsu(r_adj_g, classes=4)[0]
        r_mo1 = r_adj_g > threshold_multiotsu(r_adj_g, classes=4)[2]

        # Remove small objects before watershed
        cfp_small = skimage.morphology.remove_small_objects(cfp_mo0, 4500)
        fr_small = skimage.morphology.remove_small_objects(fr_mo1, 50)
        r_small = skimage.morphology.remove_small_objects(r_mo1, 50)

        # Erode binary image
        selem = skimage.morphology.ball(3)
        cfp_bw_opened = skimage.morphology.binary_opening(cfp_small, selem)

        # Watershed CFP and remove small objects
        label_image_cfp, cfp_dist, cfp_mrkr = ws_nuc(cfp_bw_opened)
        label_image_cfp_small2, df_cfp2, cfp_area2 = remove_small_df(label_image_cfp, 4500, cfp)
        label_image_cfp_small2_list.append(label_image_cfp_small2)
        
        # Dilate image
        selem = skimage.morphology.ball(5)
        label_image_cfp_small_dil = skimage.morphology.dilation(label_image_cfp_small2, selem)
        label_image_cfp_small_dil_list.append(label_image_cfp_small_dil)
        
        # Watershed far-red and red dots
        label_image_fr, dist_fr, mrkr_fr = ws_dot(fr_small)
        label_image_r, dist_r, mrkr_r = ws_dot(r_small)

        label_image_fr_small, df_fr, fr_area = remove_small_df(label_image_fr, 50, fr)
        label_image_r_small, df_r, r_area = remove_small_df(label_image_r, 50, r)
        label_image_fr_small_list.append(label_image_fr_small)
        label_image_r_small_list.append(label_image_r_small)
        
        
        # Make an OR mask
        r_c0 = df_r['centroid-0'].values
        r_c1 = df_r['centroid-1'].values
        r_c2 = df_r['centroid-2'].values
        r_centroids = np.zeros_like(r_mo1)

        for i, (a, b, c) in enumerate(zip(r_c0, r_c1, r_c2)):
            r_centroids[a, b, c] = 1

        fr_c0 = df_fr['centroid-0'].values
        fr_c1 = df_fr['centroid-1'].values
        fr_c2 = df_fr['centroid-2'].values
        fr_centroids = np.zeros_like(fr_mo1)

        for i, (a, b, c) in enumerate(zip(fr_c0, fr_c1, fr_c2)):
            fr_centroids[a, b, c] = 1

        r_or_fr = np.logical_or(r_centroids, fr_centroids)
        pos_nuc = label_image_cfp_small_dil * r_or_fr
        labels_pos_nuc = skimage.measure.regionprops_table(pos_nuc, properties=('label','centroid'))

        df_pos_nuc = pd.DataFrame(labels_pos_nuc)
        df_pos_nuc['co_dots'] = [0 for i in range(len(df_pos_nuc))]
        df_pos_nuc['r_dots'] = [0 for i in range(len(df_pos_nuc))]
        df_pos_nuc['fr_dots'] = [0 for i in range(len(df_pos_nuc))]

        # Make a mask with only dot positive nuclei
        label_image_pos_nuc = np.zeros_like(label_image_cfp_small_dil)

        for i, label in enumerate(df_pos_nuc['label'].values):
            bool_array = label_image_cfp_small_dil == label
            single_label = bool_array * label
            label_image_pos_nuc += single_label
        label_image_pos_nuc_list.append(label_image_pos_nuc)
            
        # Classify dots
        co_dot, fr_dot = classify_dots(fr_c0, fr_c1, fr_c2, r_c0, r_c1, r_c2, 4)
        co_dot2, r_dot = classify_dots(r_c0, r_c1, r_c2, fr_c0, fr_c1, fr_c2, 4)

        if len(co_dot) != len(co_dot2):
            print('error')
        else:
            print(len(co_dot))

        # Make a dataframe of the classified dots
        df_fr_dot = make_df(fr_dot)
        df_r_dot = make_df(r_dot)
        df_co_dot = make_df(co_dot)
        df_co_dot2 = make_df(co_dot2)

        # Take only the dots that are within CFP+ nuclei
        df_fr_nuc = dots_to_nuc(df_fr_dot, fr_mo1, label_image_cfp_small_dil)
        df_r_nuc = dots_to_nuc(df_r_dot, fr_mo1, label_image_cfp_small_dil)
        df_co_nuc = dots_to_nuc(df_co_dot, fr_mo1, label_image_cfp_small_dil)
        df_co_nuc2 = dots_to_nuc(df_co_dot2, fr_mo1, label_image_cfp_small_dil)

        # reindex
        df_pos_nuc = df_pos_nuc.set_index('label')

        # Assign each co-dot to a nuclei label
        co_nuc0 = df_co_nuc['centroid-0'].values
        co_nuc1 = df_co_nuc['centroid-1'].values
        co_nuc2 = df_co_nuc['centroid-2'].values

        for i, (a, b, c) in enumerate(zip(co_nuc0, co_nuc1, co_nuc2)):
            label = label_image_cfp_small_dil[a, b, c]
            df_pos_nuc.at[label, 'co_dots'] += 1

        # Assign each red dot to a nuclei label
        r_nuc0 = df_r_nuc['centroid-0'].values
        r_nuc1 = df_r_nuc['centroid-1'].values
        r_nuc2 = df_r_nuc['centroid-2'].values

        for i, (a, b, c) in enumerate(zip(r_nuc0, r_nuc1, r_nuc2)):
            label = label_image_cfp_small_dil[a, b, c]
            df_pos_nuc.at[label, 'r_dots'] += 1

        # Assign each far-red dot to a nuclei label
        fr_nuc0 = df_fr_nuc['centroid-0'].values
        fr_nuc1 = df_fr_nuc['centroid-1'].values
        fr_nuc2 = df_fr_nuc['centroid-2'].values

        for i, (a, b, c) in enumerate(zip(fr_nuc0, fr_nuc1, fr_nuc2)):
            label = label_image_cfp_small_dil[a, b, c]
            df_pos_nuc.at[label, 'fr_dots'] += 1

        # New column with sum
        df_pos_nuc['sum'] = df_pos_nuc['co_dots'].values + df_pos_nuc['r_dots'].values + df_pos_nuc['fr_dots'].values

        co_dots_sum = df_pos_nuc['co_dots'].values.sum()
        r_dots_sum = df_pos_nuc['r_dots'].values.sum()
        fr_dots_sum = df_pos_nuc['fr_dots'].values.sum()

        df_pos_nuc_list.append(df_pos_nuc)

        df_counts = df_pos_nuc['sum'].value_counts(
        ).reset_index(
        ).rename(
            columns={"index": "sum", "sum": "count"})

        # need to add zeros for the CFP+ nuclei with no barcodes
        zeros_num = len(df_cfp2) - len(df_pos_nuc['sum'].values)
        zeros = [0 for i in range(zeros_num)]

        data = {'sum': [0], 'count': [zeros_num]}
        df_zeros = pd.DataFrame(data)

        df_counts_zero = df_counts.append(df_zeros)
        df_counts_zero_list.append(df_counts_zero)

        spikes = hv.Spikes(
            data=df_counts_zero,
            kdims=['sum'],
            vdims=['count']
        ).opts(
            width=300, 
            line_width=30, 
            xticks=[i for i in range(0, len(df_counts_zero))],
            padding=0.1,
            xlabel='Number of active sites',
            ylabel='Frequency'
        )
        spikes_list.append(spikes)
        
        dot_sum = df_pos_nuc['sum'].values.tolist()
        total = dot_sum + zeros

        silenced = (len(co_dot) - len(df_co_nuc)) / len(co_dot)
        any_barcode = df_counts['count'].values.sum()/len(total)
        one_barcode = df_counts['count'].values[0]/len(total)
        one_plus_barcode = df_counts['count'].values[1:].sum()/len(total)
        
        silenced_list.append(silenced)
        any_barcode_list.append(any_barcode)
        one_barcode_list.append(one_barcode)
        one_plus_barcode_list.append(one_plus_barcode)

In [ ]:
codots_function()

In [ ]:
spikes_list[2]

In [ ]:
df_pos_nuc_list[1]

In [ ]:
i=1
j=10
bokeh.io.show(show_six_ims(cfp_list[i][j], r_list[i][j], fr_list[i][j], 
                           label_image_cfp_small_dil_list[i][j], label_image_r_small_list[i][j], label_image_fr_small_list[i][j],
                           cmap=[None, None, None, 
                                 colorcet.b_glasbey_hv, colorcet.b_glasbey_hv, colorcet.b_glasbey_hv]))

0. has a nuclei with too many dots, need to remove
1. same. also seems like removing some dots that may be active sites... should reduce the remove_small for dots to 25 from 50.
